In [1]:
from collections.abc import Iterable
import requests
import time
import pandas as pd
import xml.etree.ElementTree as ET
from tqdm import tqdm
import os


def fetch_things(
    ids: Iterable[int], max_retries: int = 5, retries: int = 0
) -> str | None:
    url = f"https://www.boardgamegeek.com/xmlapi2/thing?id={','.join(map(str, ids))}"
    retries = 0
    while retries < max_retries:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        elif response.status_code == 202:
            print("BGG says: not ready yet (202). Waiting...")
            time.sleep(5)
            retries += 1
            return fetch_things(ids, max_retries, retries)
        else:
            print(f"Unexpected status: {response.status_code}")
            return None


def parse_things(xml_data: str) -> list:
    root = ET.fromstring(xml_data)
    records = []
    for item in root.findall("item"):
        record = {
            "id": item.get("id"),
            "type": item.get("type"),
            "name": (
                item.find("name").get("value")
                if item.find("name") is not None
                else None
            ),
            "year_published": (
                item.find("yearpublished").get("value")
                if item.find("yearpublished") is not None
                else None
            ),
            "min_players": (
                item.find("minplayers").get("value")
                if item.find("minplayers") is not None
                else None
            ),
            "max_players": (
                item.find("maxplayers").get("value")
                if item.find("maxplayers") is not None
                else None
            ),
            "playing_time": (
                item.find("playingtime").get("value")
                if item.find("playingtime") is not None
                else None
            ),
        }
        records.append(record)
    return records

In [2]:
things: str | None = fetch_things([13, 2, 3, 4, 5])

In [3]:
from xml.etree.ElementTree import Element


root: Element = ET.fromstring(things)
records: dict[str, str | int | float | None] = {}
item = root.findall("item")[0]

In [4]:
from xmltodict import parse
import json

with open("xctyi.json", "w") as f:
    data = parse(fetch_things([13]))

In [5]:
import sqlite3

conn = sqlite3.connect("bgg.db")
cursor = conn.cursor()

In [6]:
def get_language_dependence(item: dict) -> str:
    votes: list[dict[str, str]] = item["poll"][2]["results"]["result"]
    return max(votes, key=lambda x: int(x["@numvotes"]))["@value"]


In [7]:
item = data["items"]["item"]
def insert_data(item: dict) -> tuple:
    return (
        item["@id"],
        item["thumbnail"],
        item["image"],
        item["description"],
        item["yearpublished"]["@value"],
        item["minplayers"]["@value"],
        item["maxplayers"]["@value"],
        item["poll-summary"]["result"][0]["@value"],
        get_language_dependence(item),
        item["poll-summary"]["result"][1]["@value"],
        item["playingtime"]["@value"],
        item["minplaytime"]["@value"],
        item["maxplaytime"]["@value"],
        item["minage"]["@value"],
        item["@type"] == "boardgameexpansion",
        item["name"][0]["@value"],
    )

cursor.execute(
    "",
    insert_data(item),
)
conn.commit()

ProgrammingError: Incorrect number of bindings supplied. The current statement uses 0, and there are 16 supplied.

In [ ]:
result = cursor.fetchall()
print("SQLite Version is {}".format(result))

SQLite Version is []


In [8]:
parse(fetch_things([13]))

{'items': {'@termsofuse': 'https://boardgamegeek.com/xmlapi/termsofuse',
  'item': {'@type': 'boardgame',
   '@id': '13',
   'thumbnail': 'https://cf.geekdo-images.com/PyUol9QxBnZQCJqZI6bmSA__thumb/img/virV2Bm82Dql7gh-LZScBwqByik=/fit-in/200x150/filters:strip_icc()/pic8632666.png',
   'image': 'https://cf.geekdo-images.com/PyUol9QxBnZQCJqZI6bmSA__original/img/g11AF48C6pLizxWPAq9dUEeKltQ=/0x0/filters:format(png)/pic8632666.png',
   'name': [{'@type': 'primary', '@sortindex': '1', '@value': 'CATAN'},
    {'@type': 'alternate', '@sortindex': '1', '@value': 'Catan'},
    {'@type': 'alternate',
     '@sortindex': '1',
     '@value': 'Catan (Колонизаторы)'},
    {'@type': 'alternate', '@sortindex': '1', '@value': 'Catan telepesei'},
    {'@type': 'alternate', '@sortindex': '1', '@value': 'Catan: Das Spiel'},
    {'@type': 'alternate', '@sortindex': '1', '@value': 'Catan: Die Bordspel'},
    {'@type': 'alternate', '@sortindex': '1', '@value': 'Catan: El Juego'},
    {'@type': 'alternate',
   